#### Context manager with generator function

In [1]:
def my_gen():
    try:
        print("Creating context and  yielding object")
        # exception can get into the generator (will be discussed later)
        yield [1,2,3,4,5,6]
    finally:
        print("Exiting context and cleaning up")


In [2]:
gen = my_gen()
list_ = next(gen)

Creating context and  yielding object


In [3]:
list_

[1, 2, 3, 4, 5, 6]

In [4]:
try:
    next(gen)
except StopIteration as e:
    print(e)

Exiting context and cleaning up



In [5]:
gen = my_gen()
# enter the context
list_ = next(gen)
print(list_)
# exit the context
try:
    next(gen)
except StopIteration:
    pass


Creating context and  yielding object
[1, 2, 3, 4, 5, 6]
Exiting context and cleaning up


In [6]:
class GenCtxManager:
    def __init__(self, gen_func):
        self._gen = gen_func()  # generator with try - finally is expected

    def __enter__(self):
        return next(self._gen)

    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False


In [7]:
def my_gen():
    try:
        print("Creating context and  yielding object")
        # exception can get into the generator (will be discussed later)
        yield [1,2,3,4,5,6]
    finally:
        print("Exiting context and cleaning up")


In [8]:
with GenCtxManager(my_gen) as obj:
    print(obj)

Creating context and  yielding object
[1, 2, 3, 4, 5, 6]
Exiting context and cleaning up


In [9]:
class GenCtxManager:
    def __init__(self, gen_func, *args, **kwargs):
        self._gen = gen_func(*args, **kwargs)  # generator with try - finally is expected

    def __enter__(self):
        return next(self._gen)

    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False

In [10]:
def open_file(fname, mode = "r"):
    f = open(fname, mode)
    try:
        print("Yielding the file")
        yield f
    finally:
        print("Closing the file")
        f.close()

In [11]:
with GenCtxManager(open_file, "test.txt", "w") as f:
    f.write("Writing from context manager :)")
    

Yielding the file
Closing the file


In [12]:
with GenCtxManager(open_file, "test.txt") as f:
    print(f.readlines())

Yielding the file
['Writing from context manager :)']
Closing the file


#### Contextmanager decorator
Generator has to follow try / finally format!

In [13]:
def open_file(fname, mode="r"):
    print("Opening the file")
    f = open(fname, mode)
    try:
        yield f
    finally:
        print("Closing the file")
        f.close()

In [14]:
class GenContextManager:
    def __init__(self, gen_instance):
        self.gen = gen_instance

    def __enter__(self):
        print("__enter__")
        return next(self.gen)

    def __exit__(self, exc_type, exc_value, exc_tb):
        print("__exit__")
        try:
            next(self.gen)
        except StopIteration:
            pass
        return False


In [15]:
file_gen = open_file("test.txt", "w")
with GenContextManager(file_gen) as f:
    f.write("Writing lines from context manager!")


__enter__
Opening the file
__exit__
Closing the file


In [16]:
file_gen = open_file("test.txt")
with GenContextManager(file_gen) as f:
    print(f.readlines())

__enter__
Opening the file
['Writing lines from context manager!']
__exit__
Closing the file


In [17]:
# now let's try to create context manager as decorator

from functools import wraps


def context_manager_dec(gen_fn):
    @wraps(gen_fn)
    def wrapper(*args, **kwargs):
        gen_instance = gen_fn(*args, **kwargs)  # instance of the gen function
        ctx = GenContextManager(gen_instance)
        return ctx
    return wrapper


In [18]:
# open_file = context_manager_dec(open_file)

@context_manager_dec
def open_file(fname, mode="r"):
    print("Opening the file")
    f = open(fname, mode)
    try:
        yield f
    finally:
        print("Closing the file")
        f.close()

In [19]:
with open_file("test.txt") as f:
    print(f.readlines())

__enter__
Opening the file
['Writing lines from context manager!']
__exit__
Closing the file


In [20]:
from contextlib import contextmanager

@contextmanager
def open_file(fname, mode="r"):
    print("Opening the file")
    f = open(fname, mode)
    try:
        yield f
    finally:
        print("Closing the file")
        f.close()

In [21]:
with open_file("test.txt") as f:
    print(f.readlines())

Opening the file
['Writing lines from context manager!']
Closing the file


In [22]:
from time import perf_counter, sleep


@contextmanager
def timer():
    stats = {"start": perf_counter()}
    try:
        yield stats
    finally:
        stats["end"] = perf_counter()
        stats["elapsed"] = stats["end"] - stats["start"]
        

In [23]:
with timer() as stats:
    sleep(2)

In [24]:
stats

{'start': 79258.28944308, 'end': 79260.293438951, 'elapsed': 2.003995871011284}

In [25]:
import sys


@contextmanager
def out_to_file(fname: str):
    current_stdout = sys.stdout
    file = open(fname, "w")
    sys.stdout = file
    try:
        yield None
    finally:
        sys.stdout = current_stdout
        file.close()
    

In [26]:
with out_to_file("std-to-file.txt"):
    print("stdout 1")
    print("stdout 2")

In [27]:
with open_file("std-to-file.txt") as f:
    print(f.readlines())

Opening the file
['stdout 1\n', 'stdout 2\n']
Closing the file


In [28]:
from contextlib import redirect_stdout

help(redirect_stdout)

Help on class redirect_stdout in module contextlib:

class redirect_stdout(_RedirectStream)
 |  redirect_stdout(new_target)
 |
 |  Context manager for temporarily redirecting stdout to another file.
 |
 |  # How to send help() to stderr
 |  with redirect_stdout(sys.stderr):
 |      help(dir)
 |
 |  # How to write help() to a file
 |  with open('help.txt', 'w') as f:
 |      with redirect_stdout(f):
 |          help(pow)
 |
 |  Method resolution order:
 |      redirect_stdout
 |      _RedirectStream
 |      AbstractContextManager
 |      abc.ABC
 |      builtins.object
 |
 |  Data and other attributes defined here:
 |
 |  __abstractmethods__ = frozenset()
 |
 |  ----------------------------------------------------------------------
 |  Methods inherited from _RedirectStream:
 |
 |  __enter__(self)
 |      Return `self` upon entering the runtime context.
 |
 |  __exit__(self, exctype, excinst, exctb)
 |      Raise any exception triggered within the runtime context.
 |
 |  __init__(self, 

In [29]:
with open("std-to-file.txt", "w") as f:
    with redirect_stdout(f):
        print("I'm redirected to a file!")


In [30]:
with open("std-to-file.txt") as f:
    print(f.readlines())

["I'm redirected to a file!\n"]
